# Описание

В этом ноутбуке мы обучаем модель Roberta на отзывах об отелях, тестируем на них же. Сохраняем модель в google drive, затем в качестве проверки загружаем эту модель из google drive и убеждаемся, что результаты тестирования для нее точно такие же, как у непосредственно обученной в этом ноутбуке модели.

In [ ]:
MAX_LEN = 128 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 58.3 MB/s 
     |████████████████████████████████| 7.6 MB 53.3 MB/s 


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 6.0 MB 30.6 MB/s 
     |████████████████████████████████| 439 kB 50.6 MB/s 
     |████████████████████████████████| 1.7 MB 50.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
max_len = MAX_LEN
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.BinaryCrossentropy()

input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
output = roberta_model.roberta([input_ids,attention_masks])
output = output[1]
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)
model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=opt, loss=loss, metrics=METRICS)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.drop(columns=['Rating'], inplace=True)
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Урезаем отзывы

parameter_const = 3 # подобрали
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка
n_test = (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)
df_balanced_test = pd.concat([df_negative_test, df_positive_test])
X_test = df_balanced_test['DATA_COLUMN'].squeeze().values
y_test = df_balanced_test['LABEL_COLUMN'].squeeze().values
# Обучающая выборка
n_train_neg = df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train_neg)
n_train_pos = df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train_pos)
df_balanced_train = pd.concat([df_negative_train, df_positive_train])
X_train = df_balanced_train['DATA_COLUMN'].squeeze().values
y_train = df_balanced_train['LABEL_COLUMN'].squeeze().values


In [ ]:
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# код ниже нужен для проверки

In [ ]:
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 104


In [ ]:
for txt in X_test:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 120


In [ ]:
def tokenize_roberta(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [ ]:
history_2 = model.fit([train_input_ids,train_attention_masks], y_train, epochs=5, batch_size=30)

Epoch 1/5
52/52 [==============================] - 2224s 42s/step - loss: 0.4981 - accuracy: 0.7662 - precision: 0.7679 - recall: 0.7414
Epoch 2/5
52/52 [==============================] - 2155s 41s/step - loss: 0.1693 - accuracy: 0.9454 - precision: 0.9339 - recall: 0.9549
Epoch 3/5
52/52 [==============================] - 2178s 42s/step - loss: 0.1049 - accuracy: 0.9647 - precision: 0.9533 - recall: 0.9748
Epoch 4/5
52/52 [==============================] - 2157s 41s/step - loss: 0.0618 - accuracy: 0.9801 - precision: 0.9763 - recall: 0.9828
Epoch 5/5
35/52 [===================>..........] - ETA: 11:47 - loss: 0.0474 - accuracy: 0.9829 - precision: 0.9766 - recall: 0.9882

In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/roberta_trained_on_hotel_reviews_21_december'
model.save(saved_model_path, include_optimizer=True) 


In [ ]:
result_roberta = model.predict([test_input_ids,test_attention_masks])


146/146 [==============================] - 2054s 14s/step


In [ ]:
result_roberta

array([[2.2844177e-04],
       [2.8542813e-04],
       [3.1124309e-04],
       ...,
       [9.7705162e-01],
       [9.9728757e-01],
       [9.9883980e-01]], dtype=float32)

In [ ]:
y_predicted_roberta = np.where(result_roberta > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
df_roberta_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])
df_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted_roberta)

In [ ]:
df_roberta_for_reviews_on_hotels

,accuracy,precision,recall,f1_score
reviews on hotels,0.925712,0.93189,0.913351,0.922527


In [ ]:
from keras.models import load_model
loaded_model = load_model(saved_model_path)
loaded_result_roberta = loaded_model.predict([test_input_ids,test_attention_masks])
y_predicted_roberta_loaded = np.where(loaded_result_roberta > 0.5, 1, 0)

df_loaded_roberta_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])
df_loaded_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted_roberta_loaded)
df_loaded_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted_roberta_loaded)
df_loaded_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted_roberta_loaded)
df_loaded_roberta_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted_roberta_loaded)


146/146 [==============================] - 2010s 14s/step


In [ ]:
df_loaded_roberta_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.925712,0.93189,0.913351,0.922527


Убедились, что результаты accuracy, precision, recall, f1_score у модели, обученной в этом ноутбуке, и у модели, загруженной из google drive (это та же самая модель), совпали.